In [ ]:
import sqlite3
import pandas as pd
from google.colab import files

# 1) CSV を読み込む
nba_df = pd.read_csv('/content/NBA_Stats.csv')
mvp_df = pd.read_csv('/content/MVP_Stats.csv')

# 2) メモリ上に SQLite を作成し、データを書き込む
conn = sqlite3.connect(':memory:')
nba_df.to_sql('nba_stats', conn, if_exists='replace', index=False)
mvp_df.to_sql('mvp_stats', conn, if_exists='replace', index=False)

# 3) LEFT JOIN で Share を結合し、MVP_Nominated を CASE で作成
query = """
SELECT
  n.*,
  COALESCE(m.Share, 0) AS Share,
  CASE
    WHEN COALESCE(m.Share, 0) > 0 THEN 1
    ELSE 0
  END AS MVP_Nominated
FROM nba_stats AS n
LEFT JOIN mvp_stats AS m
  ON n.Player = m.Player;
"""
merged_df = pd.read_sql_query(query, conn)


# 5) CSV として出力し、ダウンロード
merged_df.to_csv('NBA_Merged.csv', index=False)
files.download('NBA_Merged.csv')

In [ ]:
# 1) CSV を読み込む
nba_df = pd.read_csv('/content/NBA_Merged.csv')

# 2) メモリ上に SQLite を作成し、データを書き込む
conn = sqlite3.connect(':memory:')
merged_df.to_sql('nba_merged', conn, if_exists='replace', index=False)

query = """
SELECT
  Pos AS Position,
  CASE WHEN Share > 0 THEN 1 ELSE 0 END AS MVP_Nominated,
  ROUND("FG%" * 100, 2)  AS FG_Pct,
  ROUND("3P%" * 100, 2)  AS "3P_Pct",
  ROUND("2P%" * 100, 2)  AS "2P_Pct",
  ROUND(PTS,  2)  AS PTS,
  ROUND("3P", 2)  AS "3P",
  ROUND("2P", 2)  AS "2P",
  ROUND(AST,  2)  AS AST,
  ROUND(STL,  2)  AS STL,
  ROUND(BLK,  2)  AS BLK,
  ROUND(TRB,  2)  AS REB,
  Player,
  Season,
  ROUND(Share * 100, 2)  AS Vote_Share_Pct,
  ROUND(MP, 2)           AS MP_per_Game
FROM nba_merged
WHERE MP >= 10;
"""
merged_df = pd.read_sql_query(query, conn)

conn.close()

print(merged_df)

merged_df.to_csv('nba_filtered.csv', index=False)
files.download('nba_filtered.csv')